# Unsloth Download

In [1]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

Found existing installation: unsloth 2024.11.1
Uninstalling unsloth-2024.11.1:
  Successfully uninstalled unsloth-2024.11.1
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-lmf6od7h/unsloth_ec8e732ff4104fa1a212dd960351e6cf
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-lmf6od7h/unsloth_ec8e732ff4104fa1a212dd960351e6cf
  Resolved https://github.com/unslothai/unsloth.git to commit 3ea7044fc0770499abc2ddc19b85e1962fbbaea0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.11.1-py3-none-any.whl size=160367 sha256=1bbcd5d45c8b9fa186908ea4392ccd98972eb85c565bba22c496a3632f4e8a4f
  Stored in directory: /tmp/pip-ephem-wheel-cache-s_ui8rg2/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


# Imports

In [31]:
from unsloth import FastLanguageModel
import torch

from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

import json

# Hugginface Login

In [4]:
!huggingface-cli login # Enter your Hugging Face API token from https://huggingface.co/settings/tokens


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Model Config

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained (
    model_name = "meta-llama/Llama-3.1-8B",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2024.11.1: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

meta-llama/Llama-3.1-8B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [6]:
model = FastLanguageModel.get_peft_model (
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.11.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Load Dataset

In [7]:
dataset = load_dataset("ahmeterdempmk/E-Commerce-Fine-Tune-Data", split = "train") # Sanjay19tsh/fastFood

Generating train split:   0%|          | 0/320 [00:00<?, ? examples/s]

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame(dataset)
df.head()

,input,target_header,target_description
0,"Afyon Sucuk, traditional Turkish sausage from ...",Afyon Sucuk - Traditional Turkish Sausage,"- Experience the flavors of Afyon Sucuk, a tra..."
1,"Ağrı Balı, traditional Turkish honey from Ağrı...",Ağrı Balı - Traditional Turkish Honey,"- Indulge in the sweetness of Ağrı Balı, tradi..."
2,"Amasya Elması, traditional Turkish apple from ...",Amasya Elması - Traditional Turkish Apple,- Enjoy the crunchy texture and sweet flavor o...
3,"Ankara Tiftiği, traditional Turkish cotton fab...",Ankara Tiftiği - Traditional Turkish Cotton Fa...,- Experience the softness and durability of An...
4,"Artvin Yemişi, traditional Turkish citrus frui...",Artvin Yemişi - Traditional Turkish Citrus Fruit,- Enjoy the tangy flavor and crunchy texture o...


In [13]:
df.shape

(320, 3)

# Train Config & Training

In [14]:
trainer = SFTTrainer (
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "input",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments (
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/320 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 320 | Num Epochs = 30
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 4
\        /    Total batch size = 128 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,3.453500
2,3.431400
3,5.069100
4,3.366300
5,4.661900
6,2.958300
7,2.561900
8,3.752100
9,2.198600
10,2.903300


In [37]:
text = "pasta with tomato paste 5 $"

prompt = f"""
You are extracting Food title and description from given text and rewriting the description and enhancing it when necessary.
Always give response in the user's input language.
Always answer in the given json format. Do not use any other keywords. Do not make up anything.
The description part must be contain at least 5 sentences for each.

Json Format:
{{
"title": "<title of the Food>",
"description": "<description of the Food>"
}}
Examples:
Food Information: Rosehip Marmalade, keep it cold
Answer: {{"title": "Rosehip Marmalade", "description": "You should store this delicious rose marmalade in a cold place. It is an excellent flavor used in meals and desserts. Sold in grocery stores. It is in the form of 24 gr / 1 package. You can use this wonderful flavor in your meals and desserts!"}}
Food Information: Blackberry jam spoils in the heat
Answer: {{"title": "Blackberry Jam", "description": "Please store in a cold environment. It is recommended to be consumed for breakfast. It is very sweet. It is a traditional flavor and can be found in markets etc. You can also use it in your meals other than breakfast."}}
Now answer this:
Food Information: {text}
"""

In [38]:
print(prompt)


You are extracting Food title and description from given text and rewriting the description and enhancing it when necessary.
Always give response in the user's input language.
Always answer in the given json format. Do not use any other keywords. Do not make up anything.
The description part must be contain at least 5 sentences for each.

Json Format:
{
"title": "<title of the Food>",
"description": "<description of the Food>"
}
Examples:
Food Information: Rosehip Marmalade, keep it cold
Answer: {"title": "Rosehip Marmalade", "description": "You should store this delicious rose marmalade in a cold place. It is an excellent flavor used in meals and desserts. Sold in grocery stores. It is in the form of 24 gr / 1 package. You can use this wonderful flavor in your meals and desserts!"}
Food Information: Blackberry jam spoils in the heat
Answer: {"title": "Blackberry Jam", "description": "Please store in a cold environment. It is recommended to be consumed for breakfast. It is very sweet.

In [39]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [41]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

In [42]:
output = model.generate(**inputs, max_new_tokens=500)

In [25]:
response = tokenizer.decode(output[0], skip_special_tokens=True)

In [44]:
answer_start = response.find("Now answer this:") + len("Now answer this:")
answer = response[answer_start:].strip()

In [45]:
json_start = answer.find("{")
json_end = answer.find("}") + 1
json_response = answer[json_start:json_end].strip()

In [46]:
json_response

'{"title": "Pasta with Tomato Paste, 5$, "description": "You can find this traditional Italian dish in grocery stores. It is made of pasta, tomato sauce, and cheese. It is served hot, and you can enjoy it with a glass of red wine. It is a perfect dish for a quick meal or a party."}'

In [48]:
model.save_pretrained("Llama-3.1-8B-Fast-Food-Based-Tuned")
tokenizer.save_pretrained("Llama-3.1-8B-Fast-Food-Based-Tuned")

('Llama-3.1-8B-Fast-Food-Based-Tuned/tokenizer_config.json',
 'Llama-3.1-8B-Fast-Food-Based-Tuned/special_tokens_map.json',
 'Llama-3.1-8B-Fast-Food-Based-Tuned/tokenizer.json')

In [49]:
from huggingface_hub import hf_api

In [50]:
api = hf_api.HfApi()

In [53]:
model.push_to_hub("ahmeterdempmk/Llama-3.1-8B-Fast-Food-Based-Tuned")
tokenizer.push_to_hub("ahmeterdempmk/Llama-3.1-8B-Fast-Food-Based-Tuned")

Saved model to https://huggingface.co/ahmeterdempmk/Llama-3.1-8B-Fast-Food-Based-Tuned
